# Dynet segmentation with tf fold
![animation](../../fold/tensorflow_fold/g3doc/animation.gif)  

In [1]:
#just a bunch of fun
import numpy as np
import six
import time
from multiprocessing import Process, Queue
import time
import data
import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.InteractiveSession(config=config)
import tensorflow_fold as td
from conv_lstm_cell import *

# params
EMBEDDING_SIZE = 64
SEP = "|"
BATCH_SIZE = 100
data_dir = "/home/moon/data/"

#our alphabet

vocabulary = data.vocabulary(data_dir + 'vocabulary')
vsize=len(vocabulary)
print(vocabulary)

index = lambda char: vocabulary.index(char)
char = lambda i: vocabulary[i]



[' ', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'C', 'D', 'E', 'G', 'L', 'N', 'S', 'T', 'Z', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\xa0', '²', '³', '¹', 'á', 'é', 'í', 'ó', 'ö', 'ú', 'ü', 'ő', 'ű']


In [2]:
class data():
    def __init__(self, folder):
        self.data_dir = folder
        self.data = dict()
        self.size = dict()
        self.datasets = ["train", "test", "validation"]
        
        for dataset in self.datasets:
            self.data[dataset] = self.sentence_reader(folder+dataset)
            self.size[dataset] = sum(1 for line in open(folder+dataset))

                        
    def sentence_reader(self, file):
        """
        read sentences from the data format setence: word\tword\n.....\t\n
        """
        i=0
        while True:
            sentence = []
            end_sentence = False
            with open(file) as f:
                for lines in f:
                    line = lines[:-1].split('\t')
                    if line[0] != "":
                        sentence.append(line)
                    else:
                        end_sentence = True
                    if end_sentence:
                        end_sentence = False
                        sent = " ".join([word[0] for word in sentence])
                        segmented = " ".join([word[1].replace(" ","|") for word in sentence])
                        tags = []
                        last_char = "_"
                        for char in segmented:
                            if char != "|":
                                tags.append(0 if last_char!="|" else 1)
                            last_char = char
                        if len(sent) != 0:
                            sent_onehot = self.onehot(sent)
                            yield ([sent_onehot, sent_onehot[::-1]], tags)
                            sentence = []
                        
          
            
    def onehot(self, string):
        onehot = np.zeros([len(string),vsize])
        indices = np.arange(len(string)), np.array([int(index(char)) for char in string])
        onehot[indices]=1
        return [onehot[i,:] for i in range(len(onehot))]
            
store = data("/home/moon/data/")

## helper functions

In [3]:
def params_info():
    total_parameters = 0
    for variable in tf.trainable_variables():
        # shape is an array of tf.Dimension
        shape = variable.get_shape()
        print(variable.name, shape)
        # print(len(shape))
        variable_parametes = 1
        for dim in shape:
            # print(dim)
            variable_parametes *= dim.value
        print("\tparams: ", variable_parametes)
        total_parameters += variable_parametes
    print(total_parameters)
    return total_parameters

def onehot(string):
    onehot = np.zeros([len(string),vsize])
    onehot[np.arange(len(string)), np.array([index(char) for char in string])]=1
    return [onehot[i,:] for i in range(len(onehot))]


In [ ]:
#cell = td.ScopedLayer(tf.contrib.rnn.BasicLSTMCell(num_units=16), 'char_cell')


In [5]:
def bidirectional_conv_LSTM():
    convlstm = Conv1DLSTMCell(input_shape=[vsize,1], output_channels=8, kernel_shape=[5])
    conv_lstm_cell_1d = td.ScopedLayer(convlstm)

    bidir_conv_lstm = td.Composition()
    with bidir_conv_lstm.scope():
        
        forward = td.Identity().reads(bidir_conv_lstm.input[0])
        backward = td.Identity().reads(bidir_conv_lstm.input[1])

        forw = (td.RNN(conv_lstm_cell_1d) >>
                td.GetItem(1) >>
                td.GetItem(0) >> td.Function(lambda rnn_outs: tf.contrib.layers.flatten(rnn_outs))
               ).reads(forward)

        backw = (td.RNN(conv_lstm_cell_1d) >>
                 td.GetItem(1) >>
                 td.GetItem(0) >>
                 td.Function(lambda rnn_outs: tf.contrib.layers.flatten(rnn_outs))).reads(backward)

        rnn_outs = td.Concat().reads(forw,backw)
        bidir_conv_lstm.output.reads(rnn_outs)
    return bidir_conv_lstm >> td.FC(1)


def FCNN():
    return td.FC(50) >> td.FC(1)# >> td.Function(lambda xs: tf.squeeze(xs, axis=1))

data = td.Record((td.Map(
                        td.Vector(vsize) >>
                        td.Function(lambda x: tf.reshape(x, [-1,vsize,1]))),
                  td.Map(
                        td.Vector(vsize) >>
                        td.Function(lambda x: tf.reshape(x, [-1,vsize,1])))))
bidir =  data >> bidirectional_conv_LSTM()
#fc = FCNN()
blk = bidir# >> fc 

In [ ]:
compiler = td.Compiler.create((blk, td.Scalar()))
model_output, target = compiler.output_tensors
loss = tf.nn.l2_loss(model_output - target)
opt = tf.train.AdamOptimizer(learning_rate=0.001)
train_op = opt.minimize(loss)
sess.run(tf.global_variables_initializer())



In [ ]:
sess.run(tf.global_variables_initializer())

In [ ]:
losses = []
for i in range(100):
    loff, _ = sess.run([loss, train_op], compiler.build_feed_dict([([onehot("aa"), onehot("bbf")],0.5) for _ in range(100)]))
    losses.append(loss)
    if i%10==0:
        print(loff)

In [ ]:
sess.run([model_output], compiler.build_feed_dict([[[onehot("aa"), onehot("bbf")],1] for _ in range(1)]))[0][0]

In [ ]:
!nvidia-smi

In [279]:
#reverse = (td.Map(td.Scalar()) >> Slice(step=-1))

def bi():
    out_features = 64
    convlstm = Conv1DLSTMCell(input_shape=[vsize,1], output_channels=out_features, kernel_shape=[vsize])
    conv_lstm_cell_1d = td.ScopedLayer(convlstm)

    bidir_conv_lstm = td.Composition()
    with bidir_conv_lstm.scope():        
        fw_seq = td.Identity().reads(bidir_conv_lstm.input)
        bw_seq = td.Slice(step=-1).reads(fw_seq)

        forward_dir = (td.RNN(conv_lstm_cell_1d) >> td.GetItem(0)).reads(fw_seq)

        back_dir = (td.RNN(conv_lstm_cell_1d) >> td.GetItem(0)).reads(bw_seq)
        
        back_to_leftright = td.Slice(step=-1).reads(back_dir)
        
        output_transform = (td.Function(lambda x: tf.reshape(x, [-1,vsize*out_features])) >>
                            td.FC(10) >>
                            td.FC(1))
        
        bidir_common = (td.ZipWith(td.Concat() >> 
                                  output_transform >> 
                                  td.Metric('logits'))).reads(forward_dir, back_to_leftright)
                    
        #tag_logits = td.Map(output_transform).reads(bidir_common)

        bidir_conv_lstm.output.reads(bidir_common)
    return bidir_conv_lstm


data = td.Map(td.Vector(vsize) >> td.Function(lambda x: tf.reshape(x, [-1,vsize,1])))

model =  data >> bi() >> td.Void()
labels = td.Map(td.Scalar() >> td.Metric("labels")) >> td.Void()

In [280]:
compiler = td.Compiler.create((model, labels))

loss = tf.nn.l2_loss(compiler.metric_tensors['labels'] - compiler.metric_tensors['logits'])
opt = tf.train.AdamOptimizer(learning_rate=0.001)
train_op = opt.minimize(loss)
sess.run(tf.global_variables_initializer())


/home/moon/Envs/tf1/lib/python3.5/site-packages/tensorflow/python/ops/gradients_impl.py:95: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [281]:
for i in range(100):
    a,b,c= sess.run([compiler.metric_tensors['logits'], loss, train_op],compiler.build_feed_dict([(onehot("value"),[1.0,0.0,1.0,0.0,0.0])]))
    print(a,b)

[[ 0.00220643]
 [ 0.00760484]
 [ 0.00416973]
 [ 0.00292062]
 [ 0.00477531]] 4.95692
[[ 0.02287579]
 [ 0.057783  ]
 [ 0.07244746]
 [ 0.09886152]
 [ 0.11806049]] 4.342
[[ 0.04844023]
 [ 0.11468513]
 [ 0.15359043]
 [ 0.20911664]
 [ 0.25825381]] 3.80561
[[ 0.07689697]
 [ 0.18113592]
 [ 0.25709108]
 [ 0.36086401]
 [ 0.47587466]] 3.45002
[[ 0.10772377]
 [ 0.25519133]
 [ 0.3773036 ]
 [ 0.53262156]
 [ 0.75097859]] 3.61921
[[ 0.1256361 ]
 [ 0.28107011]
 [ 0.38721466]
 [ 0.47888273]
 [ 0.6281032 ]] 3.36959
[[ 0.14048012]
 [ 0.29696044]
 [ 0.382002  ]
 [ 0.41087475]
 [ 0.51979142]] 3.2319
[[ 0.15524819]
 [ 0.31215006]
 [ 0.38015923]
 [ 0.35987121]
 [ 0.45867768]] 3.18267
[[ 0.17076042]
 [ 0.32905355]
 [ 0.38440377]
 [ 0.32592076]
 [ 0.42950031]] 3.16046
[[ 0.18716422]
 [ 0.34842092]
 [ 0.39514261]
 [ 0.30741882]
 [ 0.42257553]] 3.14266
[[ 0.20429911]
 [ 0.37006199]
 [ 0.41148293]
 [ 0.30194876]
 [ 0.43096167]] 3.12475
[[ 0.22180177]
 [ 0.39293176]
 [ 0.43104634]
 [ 0.3056289 ]
 [ 0.44788119]] 3.1

[[ 0.40004751]
 [ 0.39978489]
 [ 0.39907438]
 [ 0.39965689]
 [ 0.39947927]] 3.0
[[ 0.3999595 ]
 [ 0.40018547]
 [ 0.39986482]
 [ 0.40016884]
 [ 0.40049672]] 3.0


In [282]:
sess.run(tf.global_variables_initializer())